# Sample Databricks Assistant Skill

This notebook demonstrates how to create a custom Databricks Assistant Skill. Skills are modular knowledge packages that extend the Assistant's capabilities in specific domains.

## What is a Skill?

A Skill is a collection of:
* **SKILL.md** - Main documentation with YML frontmatter defining scope and guidance
* **README.md** - Overview and usage instructions
* **Code examples** - Reusable code snippets and patterns
* **Scripts** - Pre-tested utility scripts for common tasks

## Skill Structure

```
skills/
└── data-quality-checks/
    ├── SKILL.md              # Main skill file with YML frontmatter
    ├── README.md             # Overview and instructions
    ├── examples/
    │   ├── basic-checks.py   # Code examples
    │   └── advanced-checks.py
    └── scripts/
        └── run-checks.sh     # Executable scripts
```

## Example Use Case: Data Quality Checks

This sample demonstrates a **Data Quality Checks** skill that helps users validate data completeness, detect duplicates, and implement quality monitoring.

## SKILL.md File

The SKILL.md file is the core of your skill. It contains YML frontmatter and detailed guidance.

---

```markdown
---
name: data-quality-checks
version: 1.0.0
description: |
  Comprehensive data quality validation for Delta tables. Use when users need to:
  - Validate data completeness and accuracy
  - Check for null values, duplicates, or outliers
  - Implement data quality rules and constraints
  - Generate data quality reports
  - Set up automated quality monitoring
author: Field Engineering
tags:
  - data-quality
  - validation
  - delta-lake
  - monitoring
related_skills:
  - data-sampling
  - writing-sql
last_updated: 2025-02-10
---

# Data Quality Checks Skill

## Scope

This skill provides guidance for implementing comprehensive data quality checks on Delta tables in Databricks.

## When to Use This Skill

Load this skill when:
* User asks to validate data quality
* User needs to check for missing or invalid data
* User wants to implement data quality rules
* User needs to generate quality reports
* User is setting up data quality monitoring

## Core Principles

1. **Start with Schema Validation**: Always verify schema before data validation
2. **Use Delta Lake Features**: Leverage CHECK constraints and expectations
3. **Incremental Validation**: For large tables, validate incrementally
4. **Document Quality Rules**: Keep quality rules in version control
5. **Automate Monitoring**: Set up alerts for quality issues

## Common Patterns

### Pattern 1: SQL Completeness Checks

```sql
-- Check for null values in critical columns
SELECT 
  'user_id' as column_name,
  COUNT(*) as total_rows,
  SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) as null_count,
  ROUND(100.0 * SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as null_percentage
FROM catalog.schema.users

UNION ALL

SELECT 
  'email' as column_name,
  COUNT(*) as total_rows,
  SUM(CASE WHEN email IS NULL THEN 1 ELSE 0 END) as null_count,
  ROUND(100.0 * SUM(CASE WHEN email IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as null_percentage
FROM catalog.schema.users;
```

### Pattern 2: Duplicate Detection in SQL

```sql
-- Find duplicate records based on key columns
WITH duplicates AS (
  SELECT 
    user_id,
    email,
    COUNT(*) as duplicate_count
  FROM catalog.schema.users
  GROUP BY user_id, email
  HAVING COUNT(*) > 1
)
SELECT 
  d.user_id,
  d.email,
  d.duplicate_count,
  u.*
FROM duplicates d
JOIN catalog.schema.users u 
  ON d.user_id = u.user_id 
  AND d.email = u.email
ORDER BY d.duplicate_count DESC, d.user_id;
```

### Pattern 3: Delta Lake CHECK Constraints

```sql
-- Add CHECK constraints to enforce data quality at write time
ALTER TABLE catalog.schema.users 
ADD CONSTRAINT valid_email CHECK (email LIKE '%@%.%');

ALTER TABLE catalog.schema.transactions 
ADD CONSTRAINT positive_amount CHECK (amount > 0);

ALTER TABLE catalog.schema.events 
ADD CONSTRAINT valid_date CHECK (event_date <= current_date());

-- View existing constraints
SHOW TBLPROPERTIES catalog.schema.users;
```

### Pattern 4: Data Quality Summary Report

```sql
-- Comprehensive quality metrics for a table
WITH table_stats AS (
  SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT user_id) as unique_users,
    SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) as null_user_ids,
    SUM(CASE WHEN email IS NULL THEN 1 ELSE 0 END) as null_emails,
    SUM(CASE WHEN created_date IS NULL THEN 1 ELSE 0 END) as null_dates,
    MIN(created_date) as earliest_date,
    MAX(created_date) as latest_date
  FROM catalog.schema.users
)
SELECT 
  total_rows,
  unique_users,
  ROUND(100.0 * unique_users / total_rows, 2) as uniqueness_pct,
  null_user_ids,
  ROUND(100.0 * null_user_ids / total_rows, 2) as null_user_id_pct,
  null_emails,
  ROUND(100.0 * null_emails / total_rows, 2) as null_email_pct,
  null_dates,
  earliest_date,
  latest_date,
  DATEDIFF(latest_date, earliest_date) as date_range_days
FROM table_stats;
```

## Best Practices

1. **Incremental Validation for Large Tables**
   * Use partitioning to validate data incrementally
   * Focus on recent data for time-series tables
   * Sample large tables for exploratory checks

2. **Use Delta Lake Features**
   * Implement CHECK constraints for critical rules
   * Use DESCRIBE DETAIL to track table metrics
   * Leverage Delta Lake history for quality trends

3. **Document Quality Rules**
   * Keep quality rules in version control
   * Document business logic behind each rule
   * Version your quality checks

4. **Automate Monitoring**
   * Schedule quality checks as workflows
   * Set up alerts for quality threshold violations
   * Track quality metrics over time

## Common Pitfalls

* **Avoid**: Running full table scans on large tables without sampling
* **Avoid**: Implementing overly strict constraints that block valid data
* **Avoid**: Checking quality only at ingestion (validate throughout pipeline)
* **Avoid**: Ignoring data types when checking ranges and formats

## Related Documentation

* [Delta Lake Constraints](https://docs.databricks.com/delta/constraints.html)
* [Data Quality Monitoring](https://docs.databricks.com/lakehouse-monitoring/index.html)
* [Databricks SQL Functions](https://docs.databricks.com/sql/language-manual/sql-ref-functions.html)
```

## README.md File

The README provides a quick overview and usage instructions.

---

```markdown
# Data Quality Checks Skill

## Overview

This skill provides comprehensive guidance for implementing data quality validation on Delta tables in Databricks. It includes patterns for completeness checks, duplicate detection, constraint enforcement, and automated monitoring.

## Quick Start

1. **Load the Skill**: The Assistant will automatically load this skill when you ask about data quality validation
2. **Review Patterns**: Check the SKILL.md file for common validation patterns
3. **Run Examples**: Use the SQL queries in the examples section
4. **Customize**: Adapt the patterns to your specific data quality requirements

## What's Included

* **SKILL.md**: Main skill documentation with YML frontmatter
* **SQL Examples**: Completeness checks, duplicate detection, constraint management
* **Best Practices**: Guidelines for implementing quality checks at scale
* **Automation Scripts**: Sample workflows for scheduled quality monitoring

## Use Cases

* Validate data completeness (null checks)
* Detect duplicate records
* Enforce data constraints (CHECK constraints)
* Generate quality reports
* Set up automated quality monitoring
* Track quality metrics over time

## Prerequisites

* Databricks Runtime 13.3 LTS or higher
* Access to Unity Catalog (for constraint management)
* SQL Warehouse or Cluster with appropriate permissions

## Example Usage

### Check for Null Values

```sql
SELECT 
  'user_id' as column_name,
  COUNT(*) as total_rows,
  SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) as null_count
FROM catalog.schema.customers;
```

### Find Duplicates

```sql
SELECT 
  customer_id,
  COUNT(*) as duplicate_count
FROM catalog.schema.customers
GROUP BY customer_id
HAVING COUNT(*) > 1;
```

### Add Quality Constraints

```sql
ALTER TABLE catalog.schema.customers
ADD CONSTRAINT valid_email CHECK (email LIKE '%@%.%');
```

## File Structure

```
skills/data-quality-checks/
├── SKILL.md                    # Main skill file with YML frontmatter
├── README.md                   # This file
├── examples/
│   ├── completeness-checks.sql # Null value detection queries
│   ├── duplicate-detection.sql # Duplicate finding queries
│   └── quality-report.sql      # Comprehensive quality reports
└── workflows/
    └── daily-quality-check.yml # Sample workflow definition
```

## Related Skills

* **data-sampling**: Best practices for sampling data before validation
* **writing-sql**: SQL patterns for quality checks
* **using-metric-views**: Creating reusable quality metrics

## Contributing

To improve this skill:
1. Add new validation patterns to SKILL.md
2. Include SQL examples in the examples/ directory
3. Update the version number in the YML frontmatter
4. Document any new dependencies or prerequisites

## Version History

* **1.0.0** (2025-02-10): Initial release with SQL-based quality checks

## Support

For questions or issues with this skill, contact the Field Engineering team or submit feedback through the Assistant.
```

In [0]:
-- examples/completeness-checks.sql
-- Check for null values across multiple columns

-- Method 1: Individual column checks
SELECT 
  'user_id' as column_name,
  COUNT(*) as total_rows,
  SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) as null_count,
  ROUND(100.0 * SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as null_percentage,
  CASE 
    WHEN SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) = 0 THEN 'PASS'
    ELSE 'FAIL'
  END as status
FROM samples.tpch.customer

UNION ALL

SELECT 
  'c_name' as column_name,
  COUNT(*) as total_rows,
  SUM(CASE WHEN c_name IS NULL THEN 1 ELSE 0 END) as null_count,
  ROUND(100.0 * SUM(CASE WHEN c_name IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as null_percentage,
  CASE 
    WHEN SUM(CASE WHEN c_name IS NULL THEN 1 ELSE 0 END) = 0 THEN 'PASS'
    ELSE 'FAIL'
  END as status
FROM samples.tpch.customer

UNION ALL

SELECT 
  'c_phone' as column_name,
  COUNT(*) as total_rows,
  SUM(CASE WHEN c_phone IS NULL THEN 1 ELSE 0 END) as null_count,
  ROUND(100.0 * SUM(CASE WHEN c_phone IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as null_percentage,
  CASE 
    WHEN SUM(CASE WHEN c_phone IS NULL THEN 1 ELSE 0 END) = 0 THEN 'PASS'
    ELSE 'FAIL'
  END as status
FROM samples.tpch.customer;

In [0]:
-- examples/duplicate-detection.sql
-- Find duplicate records based on key columns

-- Method 1: Simple duplicate count
WITH duplicate_summary AS (
  SELECT 
    c_custkey,
    COUNT(*) as record_count
  FROM samples.tpch.customer
  GROUP BY c_custkey
  HAVING COUNT(*) > 1
)
SELECT 
  COUNT(*) as total_duplicate_groups,
  SUM(record_count) as total_duplicate_records
FROM duplicate_summary;

-- Method 2: Detailed duplicate analysis
-- WITH duplicates AS (
--   SELECT 
--     c_custkey,
--     c_name,
--     c_phone,
--     COUNT(*) as duplicate_count,
--     ROW_NUMBER() OVER (PARTITION BY c_custkey ORDER BY c_name) as row_num
--   FROM samples.tpch.customer
--   GROUP BY c_custkey, c_name, c_phone
--   HAVING COUNT(*) > 1
-- )
-- SELECT 
--   c_custkey,
--   c_name,
--   c_phone,
--   duplicate_count,
--   CASE WHEN row_num = 1 THEN 'KEEP' ELSE 'REMOVE' END as recommendation
-- FROM duplicates
-- ORDER BY duplicate_count DESC, c_custkey;

-- Method 3: Find duplicates with all original records
-- SELECT 
--   c.*,
--   COUNT(*) OVER (PARTITION BY c.c_custkey) as duplicate_count
-- FROM samples.tpch.customer c
-- QUALIFY duplicate_count > 1
-- ORDER BY c.c_custkey;

In [0]:
-- examples/quality-report.sql
-- Comprehensive data quality report

WITH table_stats AS (
  SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT c_custkey) as unique_customers,
    
    -- Null counts
    SUM(CASE WHEN c_custkey IS NULL THEN 1 ELSE 0 END) as null_custkey,
    SUM(CASE WHEN c_name IS NULL THEN 1 ELSE 0 END) as null_name,
    SUM(CASE WHEN c_phone IS NULL THEN 1 ELSE 0 END) as null_phone,
    SUM(CASE WHEN c_acctbal IS NULL THEN 1 ELSE 0 END) as null_balance,
    
    -- Value ranges
    MIN(c_acctbal) as min_balance,
    MAX(c_acctbal) as max_balance,
    AVG(c_acctbal) as avg_balance,
    
    -- Data freshness (if date column exists)
    CURRENT_DATE() as report_date
    
  FROM samples.tpch.customer
)
SELECT 
  report_date,
  total_rows,
  unique_customers,
  ROUND(100.0 * unique_customers / total_rows, 2) as uniqueness_pct,
  
  -- Completeness metrics
  null_custkey,
  ROUND(100.0 * null_custkey / total_rows, 2) as null_custkey_pct,
  null_name,
  ROUND(100.0 * null_name / total_rows, 2) as null_name_pct,
  null_phone,
  ROUND(100.0 * null_phone / total_rows, 2) as null_phone_pct,
  null_balance,
  ROUND(100.0 * null_balance / total_rows, 2) as null_balance_pct,
  
  -- Value statistics
  ROUND(min_balance, 2) as min_balance,
  ROUND(max_balance, 2) as max_balance,
  ROUND(avg_balance, 2) as avg_balance,
  
  -- Overall quality score (100 - average null percentage)
  ROUND(100 - (
    (null_custkey + null_name + null_phone + null_balance) * 100.0 / (total_rows * 4)
  ), 2) as overall_quality_score
  
FROM table_stats;

## Constraint Management Examples

### SQL Examples for Managing Delta Lake CHECK Constraints

```sql
-- examples/constraint-management.sql
-- Managing Delta Lake CHECK constraints for data quality

-- Example 1: Add email validation constraint
ALTER TABLE catalog.schema.users
ADD CONSTRAINT valid_email CHECK (email LIKE '%@%.%');

-- Example 2: Add positive amount constraint
ALTER TABLE catalog.schema.transactions
ADD CONSTRAINT positive_amount CHECK (amount > 0);

-- Example 3: Add date range constraint
ALTER TABLE catalog.schema.events
ADD CONSTRAINT valid_date CHECK (event_date <= current_date());

-- Example 4: Add status validation constraint
ALTER TABLE catalog.schema.orders
ADD CONSTRAINT valid_status 
CHECK (status IN ('pending', 'processing', 'completed', 'cancelled'));

-- Example 5: Add age range constraint
ALTER TABLE catalog.schema.customers
ADD CONSTRAINT valid_age CHECK (age >= 0 AND age <= 120);

-- View existing constraints on a table
SHOW TBLPROPERTIES catalog.schema.table;

-- Drop a constraint if needed
ALTER TABLE catalog.schema.table
DROP CONSTRAINT constraint_name;
```

### Testing Constraints

```sql
-- Test constraint violation (will fail if constraint exists)
-- This INSERT will be rejected if the valid_email constraint is active
INSERT INTO catalog.schema.users (id, email) 
VALUES (1, 'invalid-email');

-- This INSERT will succeed
INSERT INTO catalog.schema.users (id, email) 
VALUES (1, 'valid@example.com');
```

## Workflow Configuration for Automated Quality Checks

Example workflow YAML for scheduled data quality monitoring:

```yaml
# workflows/daily-quality-check.yml
# Databricks Workflow for automated data quality monitoring

name: Daily Data Quality Checks

triggers:
  - schedule:
      quartz_cron_expression: "0 0 8 * * ?"  # Run daily at 8 AM UTC
      timezone_id: "UTC"
      pause_status: "UNPAUSED"

tasks:
  - task_key: completeness_check
    description: "Check for null values in critical columns"
    sql_task:
      warehouse_id: "{{ warehouse_id }}"
      query:
        query: |
          SELECT 
            'completeness_check' as check_type,
            CURRENT_TIMESTAMP() as check_time,
            column_name,
            total_rows,
            null_count,
            null_percentage,
            status
          FROM (
            SELECT 
              'user_id' as column_name,
              COUNT(*) as total_rows,
              SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) as null_count,
              ROUND(100.0 * SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as null_percentage,
              CASE WHEN SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) = 0 THEN 'PASS' ELSE 'FAIL' END as status
            FROM catalog.schema.users
          )
          WHERE status = 'FAIL';
    
  - task_key: duplicate_check
    description: "Detect duplicate records"
    depends_on:
      - task_key: completeness_check
    sql_task:
      warehouse_id: "{{ warehouse_id }}"
      query:
        query: |
          WITH duplicates AS (
            SELECT 
              user_id,
              COUNT(*) as duplicate_count
            FROM catalog.schema.users
            GROUP BY user_id
            HAVING COUNT(*) > 1
          )
          SELECT 
            'duplicate_check' as check_type,
            CURRENT_TIMESTAMP() as check_time,
            COUNT(*) as duplicate_groups,
            SUM(duplicate_count) as total_duplicates,
            CASE WHEN COUNT(*) = 0 THEN 'PASS' ELSE 'FAIL' END as status
          FROM duplicates;

email_notifications:
  on_failure:
    - user_name@databricks.com

max_concurrent_runs: 1
timeout_seconds: 3600

tags:
  RemoveAfter: "20260210"  # Per FE workspace policy
  purpose: "data-quality-monitoring"
```

### Deployment Options

**Via Databricks UI**:
* Navigate to Workflows → Create Job
* Add SQL tasks manually
* Configure schedule and notifications

**Via Databricks CLI**:
```bash
databricks jobs create --json-file workflows/daily-quality-check.yml
```

**Best Practices**:
* Use **RemoveAfter** tags (FE workspace requirement)
* Use shared SQL warehouses (dbdemos-shared-endpoint)
* Set auto-stop to 10-30 minutes
* Schedule during off-peak hours

## How to Create Your Own Databricks Assistant Skill

### Directory Structure

```
/Workspace/Users/your.email@databricks.com/.assistant/skills/
└── your-skill-name/
    ├── SKILL.md              # Main skill file with YML frontmatter
    ├── README.md             # Quick start guide
    ├── examples/
    │   ├── example1.sql      # SQL examples
    │   └── example2.py       # Python examples
    └── workflows/
        └── automation.yml    # Workflow configs
```

### SKILL.md YML Frontmatter Template

```yaml
---
name: your-skill-name              # Lowercase with hyphens
version: 1.0.0                     # Semantic versioning
description: |                     # Multi-line description
  Use this skill when:
  - Use case 1
  - Use case 2
  - Use case 3
author: Your Name/Team
tags:
  - category1
  - category2
related_skills:
  - related-skill-1
  - related-skill-2
last_updated: 2025-02-10          # YYYY-MM-DD format
---
```

### Key Sections in SKILL.md

1. **Scope**: Define when to use this skill
2. **Core Principles**: Fundamental concepts
3. **Common Patterns**: Code examples with explanations
4. **Best Practices**: Guidelines and recommendations
5. **Common Pitfalls**: What to avoid
6. **Related Documentation**: Links to official docs

### README.md Template

```markdown
# Skill Name

## Overview
Brief description of the skill.

## Quick Start
1. When the skill is loaded
2. What it provides
3. How to use it

## What's Included
* File 1
* File 2

## Use Cases
* Use case 1
* Use case 2

## Prerequisites
* Requirement 1
* Requirement 2

## Example Usage
Code examples here

## Version History
* 1.0.0 - Initial release
```

### Best Practices

✅ **Clear Scope**: Define exactly when to use the skill  
✅ **Working Examples**: Include tested, runnable code  
✅ **Documentation Links**: Reference official Databricks docs  
✅ **Version Control**: Update version on changes  
✅ **FE Compliance**: Follow workspace policies  
✅ **Focused Domain**: Keep skills modular and specific

### How the Assistant Uses Skills

1. User asks a question matching the skill's description
2. Assistant loads SKILL.md using `readSkillFile` tool
3. Assistant applies patterns and guidance from the skill
4. Assistant adapts examples to user's specific needs
5. User receives expert-level domain-specific help

### Skill Categories (Examples)

* **Data Engineering**: ETL, Delta Lake, Streaming
* **Data Quality**: Validation, Monitoring, Constraints
* **Analytics**: SQL Patterns, Dashboards, Metrics
* **Machine Learning**: Training, MLflow, Features
* **Security**: Access Control, Encryption
* **Performance**: Optimization, Caching, Partitioning

### FE Workspace Compliance

When creating skills for FE workspaces:
* Use **RemoveAfter** tags on resources
* Prefer shared compute (dbdemos-shared-endpoint)
* Store in personal folders (/Users/your.email/)
* Follow naming conventions (avoid tmp\_, test\_)
* Clean up temporary resources

---

## What This Notebook Provides

✅ Complete **SKILL.md** with YML frontmatter  
✅ Comprehensive **README.md** template  
✅ Working **SQL examples** for data quality  
✅ **Workflow configuration** for automation  
✅ **Best practices** for skill development  
✅ **FE workspace compliance** guidelines

**Next Steps**: Adapt this template for your domain expertise by modifying the content while keeping the structure!